In [66]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [67]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-13_13-22-33_206220_72767/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-13_13-22-33_206220_72767/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-06-13_13-22-33_206220_72767', 'metrics_export_port': 61965, 'gcs_address': '127.0.0.1:61630', 'address': '127.0.0.1:61630', 'node_id': '15b44fc74b48f6e83789170df52e5b222b25f0bdfa4d9dfef02db71f'})


In [68]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


In [76]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": [0], "B": [0], "C": [0]}
    
    def invoke(self, name):
        # pretend to do some work
        time.sleep(0.5)
        
        # compute a random result and update the result
        result = random.randint(5, 25)
        self.invokers[name].append(result)
        return result
        
    def get_invoker_state(self, name):
        return self.invokers[name]
    
    def get_number_of_calls(self, name):
        return len(self.invokers[name])
    
    def get_all_invoker_state(self):
        return self.invokers

In [77]:
actor_instance = MethodStateCounter.remote()

2022-06-13 13:43:29,020	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


In [78]:
for _ in range(10):
    name = random.choice(CALLERS)
    actor_instance.invoke.remote(name)

#### 1. Get number of times an invoker name was called

In [79]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_number_of_calls.remote(name))
print(f"Number of times: {name} was called: {count}")

Number of times: A was called: 5


#### 2. Get computed values by invoker name

In [80]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_invoker_state.remote(name))
print(f"All values computed by: {name} is: {count}")

All values computed by: A is: [0, 14, 17, 25, 13]


#### 3. Get state of all invokers

In [81]:
print(f"State of all computed values for invokers: {ray.get(actor_instance.get_all_invoker_state.remote())}")

State of all computed values for invokers: {'A': [0, 14, 17, 25, 13], 'B': [0, 16], 'C': [0, 14, 5, 11, 17, 12]}


In [82]:
ray.shutdown()